In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug  9 20:02:23 2020
@author: MUSTAFAAKTAS

Modified Aug  2022
@author: cmmarellano
"""

import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import geopandas as gpd 
import geoio
import json

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\")
dataset_dir = os.path.abspath("C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\")

In [3]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [4]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [5]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\mask_rcnn_spacenet_0151\\mask_rcnn_spacenet_0151.h5")
print(COCO_MODEL_PATH)
print('- - model located')

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
    print('Specify COCO_MODEL_PATH !')
    

C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\mask_rcnn_spacenet_0151\mask_rcnn_spacenet_0151.h5
- - model located
Specify COCO_MODEL_PATH !


In [6]:
# define data locations for images and annotations
images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")
        

In [7]:
class SpaceNetConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SpaceNet"
    BACKBONE = "resnet50"
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 building

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_RATIOS = [0.25, 1, 4]

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 5  #10    #32

    USE_MINI_MASK = True
    
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 50 #500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 10 #50
    
    MAX_GT_INSTANCES=250
    DETECTION_MAX_INSTANCES=350
    
config = SpaceNetConfig()
config.display()





Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        350
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [8]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def fill_between(polygon):
    """
    Returns: a bool array
    """
    img = Image.new('1', (650, 650), False)
    ImageDraw.Draw(img).polygon(polygon, outline=True, fill=True)
    mask = np.array(img)
    return mask

In [9]:


class SpaceNetDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    
    def load_dataset(self, dataset_dir=dataset_dir, start=0, end=10):   # 1-400
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        
        self.start=start
        self.end=end
        
        # Add classes
        self.add_class("SpaceNetDataset", 1, "building")

        # define data locations for images and annotations
        images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
        annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        
#         image_id_list =[]
#         counters = 1
        for filename in os.listdir(images_dir)[start:end]:
#             print(counters)
#             print(filename)        
            id_base  = os.path.basename(filename)
            finder= 'PS-RGB_img'
            id_start = filename.find(finder)+ len(finder)
            id_end = filename.find(".png")
            image_id = filename[id_start:id_end]
            image_dir = os.path.join(images_dir,str(filename))
            ann_path  = os.path.join(annotations_dir,"label_img"+str(image_id)+".geojson")  
#             print("image id : ",image_id)
#             print("image dir : ",image_dir)
#             print("ann path : ",ann_path)
            
            self.add_image('SpaceNetDataset', image_id=image_id, path=image_dir, annotation=ann_path)
#             counters = counters+1
#             image_id_list.append(image_id)
#         print("\nimage ids loaded: ", image_id_list)
        
#         self.image_id_list = image_id_list
    
        

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
            Typically this function loads the image from a file, but
            in this case it generates the image on the fly from the
            specs in image_info.
        """
#         self.load_dataset()
#         cont_id = image_id
#         image_id = self.image_id_list[cont_id]
        
        # define data locations for images and annotations
        images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
        annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")

        # Iterate through all files in the folder to 
        #add class, images and annotaions   

        image_dir = os.path.join(dataset_dir+"\\PS-RGB_select_png\\PS-RGB_img"+str(image_id)+".png")
        im = Image.open(image_dir)
        return np.asarray(im)


        
    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """

        
        masks = np.zeros((650,650))
        bldgmask_path = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\mask_for_training\\"
        ResimPATH = bldgmask_path+"mask_img"+str(image_id)+'.png'
        RGBTIFResmi = geoio.GeoImage(ResimPATH)
        
        with open(dataset_dir+"\\building_labels_select_geojson\\label_img"+str(image_id)+".geojson") as f:
            data = json.load(f)
            allBuildings = data['features']
              
            for building in allBuildings:   
                veri = building['geometry']['coordinates'][0]
                
                tip = str(building['geometry']['type'])
                coordinates = list() 
                if tip == ('Point'):
                    continue
                    
                elif tip == ('MultiPolygon'):
                    
                    if isinstance(veri,float): continue
                        
                    kucukBinalar = (building['geometry']['coordinates'])
                    for b in range(len(kucukBinalar)):  
                        veri = kucukBinalar[b][0]
                        for i in veri:
                            xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                            xPixel = 649 if xPixel > 649 else xPixel
                            yPixel = 649 if yPixel > 649 else yPixel
                            coordinates.append((xPixel,yPixel)) 
                else:
                    if isinstance(veri,float): continue
    
                    for i in veri:
                        xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                        xPixel = 649 if xPixel > 649 else xPixel
                        yPixel = 649 if yPixel > 649 else yPixel
                        coordinates.append((xPixel,yPixel))

                maske = fill_between(coordinates)
                masks = np.dstack((masks,maske))

                
        if masks.shape != (650,650):
            masks = masks[:,:,1:]
            class_ids = np.asarray([1]*masks.shape[2])
        else:
            class_ids=np.ones((1))
            masks = masks.reshape((650,650,1))
        return masks.astype(np.bool), class_ids.astype(np.int32)
    



In [10]:
# Training dataset
dataset_train = SpaceNetDataset()
dataset_train.load_dataset(dataset_dir,0,12) 
dataset_train.prepare()

In [11]:
print("Images: {}\nClasses: {}".format(len(dataset_train.image_ids), dataset_train.class_names))

Images: 12
Classes: ['BG', 'building']


In [12]:
# Validation dataset
dataset_val = SpaceNetDataset()
dataset_val.load_dataset(dataset_dir,12,16)
dataset_val.prepare()

In [13]:
print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))
print(dataset_val.image_ids)

Images: 4
Classes: ['BG', 'building']
[0 1 2 3]


In [15]:


#####  MODEL FOR TRAINING  ##############

from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')



# Create model in training mode  -> SpaceNet config
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=200, 
            layers='all')



Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\logs\spacenet20220818T1855\mask_rcnn_spacenet_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a    

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200


KeyboardInterrupt: 

In [ ]:
# # Load and display random samples
# print(dataset_train.image_ids)
# image_ids = np.random.choice(dataset_train.image_ids,2)
# print(image_ids)
# for image_id in range(len(image_ids)):
#     print(image_id)
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


In [ ]:
# Recreate the model in inference mode
# model = modellib.MaskRCNN(mode="inference", 
#                           config=inference_config,
#                           model_dir=MODEL_DIR)


In [ ]:
# model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[
#             "mrcnn_class_logits", "mrcnn_bbox_fc",
#             "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =modellib.load_image_gt(dataset_val, inference_config, image_id) #use_mini_mask=False'



In [ ]:
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

In [ ]:
# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs=200, 
#             layers='all')

In [ ]:
# ####   DETECTION     ####
# class InferenceConfig(SpaceNetConfig):
#     GPU_COUNT = 1
#     IMAGES_PER_GPU = 1

# inference_config = InferenceConfig()

In [ ]:

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))

# results = model.detect([original_image], verbose=1)
# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())



In [ ]:

# # Compute VOC-Style mAP @ IoU=0.5
# # Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 6)
# APs = []
# for image_id in image_ids:
#     # Load image and ground truth data
#     image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config,
#                                image_id) #, use_mini_mask=False
#     molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
#     # Run object detection
#     results = model.detect([image], verbose=0)
#     r = results[0]
#     # Compute AP
#     AP, precisions, recalls, overlaps =\
#         utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
#                          r["rois"], r["class_ids"], r["scores"], r['masks'])
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))
